In [13]:
import pickle
import os 
import glob
import random
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import plyfile
import numpy as np
import open3d as o3d
import h5py
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_distances
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import open3d as o3d
from sklearn.decomposition import PCA
from scipy.spatial import distance
from sklearn.metrics.pairwise import euclidean_distances

import cv2
import numpy as np

import os.path as osp
import sys
ws_dir = '/local/home/ekoller/BT'
print(ws_dir)
sys.path.append(ws_dir)
from utils import evaluation, scan3r


data_dir ='/local/home/ekoller/R3Scan'
scenes_dir = '/local/home/ekoller/R3Scan/scenes'
#scan_id= "38770c95-86d7-27b8-8717-3485b411ddc7" #is reference scan  since it is a reference scan everything shouls be correctly hit
curr_scan_id =  "02b33dfb-be2b-2d54-92d2-cd012b2b3c40"#"02b33e01-be2b-2d54-93fb-4145a709cec5" 
new_scan_id =  "fcf66d8a-622d-291c-8429-0e1109c6bb26" #"fcf66d8a-622d-291c-8429-0e1109c6bb26"



/local/home/ekoller/BT


In [14]:
#for a given scene get the colours of the differnt object_ids
def get_id_colours(data_dir,scan_id):
    #access the mesh file to get the colour of the ids
    mesh_file = osp.join(data_dir,"scenes", scan_id, "labels.instances.annotated.v2.ply")
    ply_data = plyfile.PlyData.read(mesh_file)
    # Extract vertex data
    vertices = ply_data['vertex']
    vertex_count = len(vertices)
    
    # Initialize dictionary to store object_id -> color mappings
    object_colors = {}
    
   # Iterate through vertices
    for i in range(vertex_count):
        vertex = vertices[i]
        object_id = vertex['objectId']
        color = (vertex['red'], vertex['green'], vertex['blue'])
        
        # Check if object_id already in dictionary, otherwise initialize a Counter
        if object_id in object_colors:
            object_colors[object_id][color] += 1
        else:
            object_colors[object_id] = Counter({color: 1})
    
    # Convert Counter to dictionary with most frequent color
    for object_id, color_counter in object_colors.items():
        most_common_color = color_counter.most_common(1)[0][0]
        object_colors[object_id] = np.array(most_common_color)
    
    return object_colors

colors = get_id_colours(data_dir,curr_scan_id)
print("cur scan id colors", colors.keys())
colors = get_id_colours(data_dir,new_scan_id)
print("new scan id colors", colors.keys())

cur scan id colors dict_keys([3, 10, 17, 6, 4, 16, 2, 7, 15, 14, 1, 11, 0, 22, 8, 9, 30, 28, 12, 26, 24, 25, 5, 29, 21, 20, 13, 23, 27])
new scan id colors dict_keys([13, 20, 1, 4, 100, 6, 14, 15, 18, 29, 5, 8, 16, 105, 0])


In [15]:
def acces_predicted_objects(scan_id):
    all_centers = {}
    filename = osp.join("/media/ekoller/T7/Predicted_Centers", scan_id + ".h5")
    with h5py.File(filename, 'r') as h5file:
        for obj_id in h5file.keys():
            obj_group = h5file[obj_id]
            center = np.array(obj_group['center'])
            points = np.array(obj_group['points'])
            votes = int(obj_group['votes'][()])
            
            # Add it back to the dictionary
            all_centers[int(obj_id)] = {
                'center': center,
                'points': points,
                'votes': votes
            }
    return all_centers

acces_predicted_objects(new_scan_id)

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/media/ekoller/T7/Predicted_Centers/fcf66d8a-622d-291c-8429-0e1109c6bb26.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

this segment displays the predicted pointcloud and the ground truth point cloud

In [4]:
# create the newly predicted pointcloud
all_centers = acces_predicted_objects(data_dir, new_scan_id)
#initialize the final pointcloud

final_point_cloud = o3d.geometry.PointCloud()

#colour
colors = get_id_colours(data_dir, new_scan_id)

# Iterate over all objects in all_centers
for obj_id, data in all_centers.items():
    points = data['points']  # Get points
    color = colors(obj_id)  # Generate a unique color for this object ID
    
    # Create a point cloud for the current object's points
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(points)
    
    # Create a color array of the same size as the number of points, and assign the same color to all points
    colors = np.tile(color, (points.shape[0], 1))  # Shape (num_points, 3)
    point_cloud.colors = o3d.utility.Vector3dVector(colors)

    colors = np.asarray(point_cloud.colors)
    
    # Scale the colors by the brightness factor
    adjusted_colors = np.clip(colors * 1.3, 0, 1)  # Clip to ensure values are in the [0, 1] range
    
    # Update the point cloud's colors
    point_cloud.colors = o3d.utility.Vector3dVector(adjusted_colors)

    # Add the current object's point cloud to the final point cloud
    final_point_cloud += point_cloud



#also access the ground truth pointclouds
pklfile = osp.join('/local/home/ekoller/R3Scan', 'files', 'orig', 'data', '{}.pkl'.format(new_scan_id))

with open(pklfile, "rb") as f:
    # Load the data from the pickle file
    data = pickle.load(f)
    
    # Extract object points and IDs from the pickle data
    pointclouds = data['obj_points']
    object_ids = data['objects_id']
    
    # Create a new point cloud object for the .pkl data
    pkl_point_cloud = o3d.geometry.PointCloud()

    # Iterate over all the points and their object IDs
    for obj_id, points in zip(object_ids, pointclouds):
        # Assign a default color (e.g., black) to these objects
        color = colors(obj_id) 
        # Create a point cloud for this object's points
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(points)
        pcd.colors = o3d.utility.Vector3dVector(np.tile(color, (points.shape[0], 1)))  # Assign the color

        # Add this point cloud to the combined pkl_point_cloud
        pkl_point_cloud += pcd

colors = np.asarray(pkl_point_cloud.colors)

# Scale the colors by the brightness factor
adjusted_colors = np.clip(colors *0.7, 0, 1)  # Clip to ensure values are in the [0, 1] range

# Update the point cloud's colors
pkl_point_cloud.colors = o3d.utility.Vector3dVector(adjusted_colors)
# Now combine both point clouds (final_point_cloud and pkl_point_cloud) for visualization
combined_point_cloud = final_point_cloud + pkl_point_cloud

# Visualize both together using Open3D
o3d.visualization.draw_geometries([combined_point_cloud], window_name="Combined Point Clouds from Scan and PKL")



#o3d.visualization.draw_geometries([final_point_cloud], window_name="Combined Point Cloud")



FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/media/ekoller/T7/Predicted_Centers/fcf66d9e-622d-291c-84c2-bb23dfe31327.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)